<a href="https://colab.research.google.com/github/Poojapattu/Mentalheathchatbot-InnerBalanceHuma/blob/main/MentalHealthchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -U langchain-community


In [13]:
!pip install langchain-groq

In [14]:
!pip install pypdf

In [20]:
!pip install chromadb

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
# Import the ChatGroq class from langchain_groq
from langchain_groq import ChatGroq # This line was added

def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

def create_vector_db():
    # Use a raw string to prevent misinterpretation of backslashes
    # Update path to the correct one in Google Colab environment
    loader = PyPDFLoader(r"/content/mental_health_Document (1).pdf")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()

    print("ChromaDB created and data saved")

    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_templates = """You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
    PROMPT = PromptTemplate(template=prompt_templates, input_variables=['context', 'question'])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

def main():
    print("Initializing Chatbot...")
    llm = initialize_llm()

    db_path = "machine learning projects/mental_health/content/chroma_db"
    print(f"Checking if database path exists: {db_path}")

    if not os.path.exists(db_path):
        print("Database path does not exist, creating new vector DB...")
        vector_db = create_vector_db()
    else:
        print("Database path exists, loading vector DB...")
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    qa_chain = setup_qa_chain(vector_db, llm)

    while True:
        query = input("\nHuman: ")
        if query.lower() == "exit":
            print("Chatbot: Take care of yourself, Goodbye!")
            break
        response = qa_chain.run(query)
        print(f"Chatbot: {response}")

if __name__ == "__main__":
    main()

Initializing Chatbot...
Checking if database path exists: machine learning projects/mental_health/content/chroma_db
Database path does not exist, creating new vector DB...
ChromaDB created and data saved


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2096, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1641, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/utils.py", line 857, in async_wrapper
    response = await f(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-

In [22]:
!pip install gradio

In [23]:
!pip install -U langchain-community

In [27]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr
# Import the ChatGroq class from langchain_groq
from langchain_groq import ChatGroq

def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

def create_vector_db():
    loader = PyPDFLoader(r"/content/mental_health_Document (1).pdf")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()

    print("ChromaDB created and data saved")
    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_templates = """You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
    PROMPT = PromptTemplate(template=prompt_templates, input_variables=['context', 'question'])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

print("Initializing Chatbot...")
llm = initialize_llm()

db_path = "machine learning projects/mental_health/content/chroma_db"
print(f"Checking if database path exists: {db_path}")

if not os.path.exists(db_path):
    print("Database path does not exist, creating new vector DB...")
    vector_db = create_vector_db()
else:
    print("Database path exists, loading vector DB...")
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(user_input, history=[]):
    if not user_input.strip():
        return "Please provide a valid input", history

    response = qa_chain.run(user_input)  # Correct function call
    history.append((user_input, response))
    return "", history

with gr.Blocks(theme='Respair/Shiki@1.2.1') as app:
    chatbot = gr.ChatInterface(fn=chatbot_response, title="InnerBalanceHuma ChatBot")  # Fixed gr.ChatInterface

app.launch()


Initializing Chatbot...
Checking if database path exists: machine learning projects/mental_health/content/chroma_db
Database path does not exist, creating new vector DB...
ChromaDB created and data saved


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7ce7dca90d4afc08d9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
